# USB Inrush Current limiting
We have about 20 uF of capacitance at our input.
In order to limit the inrush current.
The USB power supply is considered an ideal voltage source.
- max 10 uF for inrush current, we have about 20 uF
- The DC resistance of the ferrite is about 0.04 - 0.05 DCR
- Add a resistor just to be sure

**Conclusion**
I don't think the inrush current will be a big problem, however if it is we can always add a small resistor of about 1-2 ohms in series (0805 pad prepared here)